In [104]:
from array import array
import math


### Prosta klasa Vector2D

In [105]:
class Vector2D:
    # Typecode to atrybut klasy, który użyjemy podczas konwersji instacji
    # Vector2D na typ bytes i z tego typu.
    typecode = "d"

    def __init__(self, x, y):
        # Konwersja na float wcześnie przechwytuje błędy.
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        # Metoda __iter__ sprawia, że obiekt jest iterowalny, co sprawia
        # że działa np. rozpakowywanie. Implementujemy to używając wyrażenia
        # generatora do generowania składników jeden po drugim.
        return (component for component in (self.x, self.y))

    def __repr__(self):
        # Metoda __repr__ buduje łańcuch wstawiając składniki przy użyciu !r
        # co sprawia, że otrzymujemy ich reprezentację. Jako, że Vector2D
        # implementuje metodę __iter__ to możemy użyć rozpakowania.
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        # Z iterowalnego obiektu łatwo stworzyć obiekt tuple w celu wyświetlenia.
        return str(tuple(self))

    def __bytes__(self):
        # Aby generować bajty konwertujemy typecode na typ bytes i przeprowadzamy
        # konkatenację z bajtami konwertowanymi z tablicy zbudowanej za pomocą
        # iteracji przez instancję.
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        # Aby szybko porównać składniki, budujemy krotki z operandów.
        # To rozwiązanie działa ale ma wadę, tj. działa również gdy porównamy
        # Vector2D z iterowalnymi o tych samych wartościach.
        return tuple(self) == tuple(other)

    def __abs__(self):
        # Po prostu moduł w przestrzeni euklidesowej.
        return math.hypot(self.x, self.y)

    def __bool__(self):
        # Jest prawdą instancja o niezerowym module.
        return bool(abs(self))


In [106]:
v1 = Vector2D(2, 3)


In [107]:
print(v1.x, v1.y)


2.0 3.0


In [108]:
x, y = v1
print(x, y)


2.0 3.0


In [109]:
v1


Vector2D(2.0, 3.0)

In [110]:
v1_clone = eval(repr(v1))
v1_clone == v1


True

In [111]:
print(v1)


(2.0, 3.0)


In [112]:
octets = bytes(v1)
octets


b'd\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@'

In [113]:
abs(v1)


3.605551275463989

In [114]:
bool(v1), bool(Vector2D(0, 0))


(True, False)

### Klasa Vector2D z metodą klasy - alternatywnym konstruktorem

In [115]:
class Vector2D:
    typecode = "d"

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (component for component in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    @classmethod  # Metoda klasy.
    def frombytes(cls, octets):  # Sama klasa jest przekazywana jako cls.
        typecode = chr(octets[0])  # Odczytujemy typecode z pierwszego bajta.
        # Tworzymy widok i rzutujemy na typecode a następnie...
        memv = memoryview(octets[1:]).cast(typecode)
        # zwracamy wektor z wartościami pochodzącymi z rozpakowanego widoku.
        return cls(*memv)


In [116]:
v2 = Vector2D.frombytes(octets)
v2


Vector2D(2.0, 3.0)

### Classmethod vs Staticmethod

Dekorator `@classmethod` jest stosowany do zmiany działania metody tak aby operowała na klasie a nie na instancji klasy.
Stąd pierwszy parametr nosi nazwę `cls`. Dekorator ten zmienia sposób wywoływania metody aby jako pierwszy argument
przyjmowała klasę. Najczęstsze zastosowanie tego dekoratora to budowanie alternatywnych konstruktorów, które inicjalizują
obiekt np. z pliku lub z bajtów. Zazwyczaj sam argument `cls` jest wywoływany w instrukcji `return` do inicjalizacji 
nowej instacji klasy.

Dekorator `@staticmethod` zmienia metodę tak aby nie otrzymywała żadnego specjalnego pierwszego argumentu. W istacie metoda
taka działa jak zwykła funkcja, która przypadkiem żyje w ciele klasy i powinna być zdefiniowana na poziomie modułu.


In [117]:
class Demo:
    @classmethod
    def classmeth(*args):
        # Zwaraca cls i argumenty pozycyjne.
        return args

    @staticmethod
    def statmeth(*args):
        # Zwraca argumenty pozycyjne.
        return args


In [118]:
Demo.classmeth()

(__main__.Demo,)

In [119]:
Demo.statmeth()

()

### Klasa Vector2D z formatowanym wyświetlaniem

In [120]:
class Vector2D:
    typecode = "d"

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (component for component in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    # def __format__(self, fmt_spec=""):
    #     # Używamy wbudowanej funkcji format aby zastosować specyfikację
    #     # formatowania fmt_spec do składników wektora.
    #     components = (format(component, fmt_spec) for component in self)
    #     return "({}, {})".format(*components)

    def angle(self):
        # Kąt w radianach.
        return math.atan2(self.x, self.y)

    # Jeżeli klasa nie implementuje metody specjalnej __format__
    # to funkcja format użyje domyślnej metody __str__ klasy do wyświetlania.
    def __format__(self, fmt_spec=""):
        # Należy uważać aby nie używac domyślnych kodów formatowania.
        # Nie jest to błąd ale może wprowadzać niejasności.
        if fmt_spec.endswith("p"):  # Współrzędne biegunowe.
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = "<{}, {}>"
        else:  # Współrzędne kartezjańskie.
            coords = self
            outer_fmt = "({}, {})"

        components = (format(coord, fmt_spec) for coord in coords)
        return outer_fmt.format(*components)


In [121]:
v3 = Vector2D(1, 2)
format(v3, ".3f"), format(v3, ".1fp")


('(1.000, 2.000)', '<2.2, 0.5>')

### Haszowalny obiekt Vector2D

Aby zapewnić haszowalność obiektu, musimy zaimplementować metody `__hash()__` i `__eq()__` oraz sprawić, że obiekt jest 
niezmienny, tj. nie możemy użyć podstawienia typu np. v.x = 1. Oczekujemy, że podstawienie spowoduje AttributeError.
Osiągamy to poprzez sprawienie, że x i y są właściwościami tylko do odczytu, używając dekoratora `@property`.

In [122]:
class Vector2D:
    typecode = "d"

    def __init__(self, x, y):
        # Używamy wiodącego podkreślenia aby atrybut stał się prywatny.
        self._x = float(x)
        self._y = float(y)

    # Dekoratory @property oznaczają metodę getter właściwości.
    @property
    def x(self):  # Metoda getter jest nazwana tak samo jak wartość, którą eksponuje.
        return self._x

    @property
    def y(self):
        return self._y

    def __hash__(self):
        # Obliczamy po prostu hash krotki współrzędnych.
        return hash((self.x, self.y))

    def __iter__(self):
        return (component for component in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def angle(self):
        return math.atan2(self.x, self.y)

    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("p"):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = "<{}, {}>"
        else:
            coords = self
            outer_fmt = "({}, {})"

        components = (format(coord, fmt_spec) for coord in coords)
        return outer_fmt.format(*components)


In [123]:
v4 = Vector2D(2, 3)
hash(v4)


8409376899596376432

In [124]:
set((Vector2D(1, 2), Vector2D(3, 4)))


{Vector2D(1.0, 2.0), Vector2D(3.0, 4.0)}

### Pozycyjne dopasowywanie wzorców

Aby sprawić, że klasa Vector2D będzie działać dla wzorców pozycyjnych, musimy dodac atrybut klasy
o nazwie `__match_args__` wyliczający atrybuty instacji w tej kolejności w jakiej będą one używane
w dopasowywaniu wzorca pozycyjnego.

In [125]:
class Vector2D:
    __match_args__ = ("x", "y")
    typecode = "d"

    def __init__(self, x, y):
        self._x = float(x)
        self._y = float(y)

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    def __hash__(self):
        return hash((self.x, self.y))

    def __iter__(self):
        return (component for component in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def angle(self):
        return math.atan2(self.x, self.y)

    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("p"):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = "<{}, {}>"
        else:
            coords = self
            outer_fmt = "({}, {})"

        components = (format(coord, fmt_spec) for coord in coords)
        return outer_fmt.format(*components)


In [126]:
def positional_pattern_demo(v: Vector2D) -> None:
    match v:
        case Vector2D(0, 0):
            print(f"{v!r} is null")
        case Vector2D(0, _):
            print(f"{v!r} is vertical")
        case Vector2D(_, 0):
            print(f"{v!r} is horizontal")
        case Vector2D(x, y) if x == y:
            print(f"{v!r} is diagonal")
        case _:
            print(f"{v!r} is awesome")


In [127]:
positional_pattern_demo(Vector2D(2, 0))

Vector2D(2.0, 0.0) is horizontal


### Oszczędzanie miejsca dzięki atrybutowi `__slots__`

Domyślnie Python przechowuje atrybuty instancji w słowniku o nazwie `__dict__`, który ma znaczny narzut na pamięć.
Jeśli jednak zdefiniujemy atrybut o nazwie `__slots__`, przechowujący sekwencję nazw atrybutów, Python użyje
alternatywnego modelu przechowywania dla atrybutów instancji. Atrybuty przechowywane w `__slots__` są przechowywane
w ukrytej tablicy lub jako referencje, które zajmują znacznie mniej miejsca niz typ `__dict__`.

In [128]:
class Pixel:
    __slots__ = ("x", "y")


In [129]:
p = Pixel()
p.__dict__


AttributeError: 'Pixel' object has no attribute '__dict__'

In [130]:
p.x = 10
p.y = 20

In [131]:
p.color = "red"  # Atrybutu nie ma w `__slots__`!


AttributeError: 'Pixel' object has no attribute 'color'

In [132]:
class OpenPixel(Pixel):
    # `OpenPixel` nie deklaruje żadnych własnych atrybutów.
    pass


In [133]:
op = OpenPixel()
op.__dict__  # Zawiera __dict__


{}

In [134]:
op.x = 8  # Jeżeli ustawimy atrybut x (nazwany w `__slots__` klasy bazowej)...
op.__dict__  # to nie jest on przechowywany w __dict__ instancji


{}

In [135]:
op.x  # ale w ukrytej tablicy referencji w tej instancji


8

In [137]:
op.color = "green"  # Jeśli jednak ustawimy atrybut nienazwany w `__slots__`
op.__dict__  # to jest on umieszczany w __dict__ instancji.


{'color': 'green'}

Efekt użycia `__slots__` jest tylko częściowo dziedziczony przez podklasę. Aby mieć pewność, że podklasy
również nie będą używać `__dict__` musimy ponownie zadeklarować `__slots__` w podklasie. 
Jeśli zadeklarujemy `__slots__ = ()`, instancje podklasy nie będą zawierać `__dict__` i będą akceptować
atrybuty nazwane jedynie w `__slots__` klasy nadrzędnej. Jeśli chcemy aby podklasa miała dodatkowe atrybuty,
nazywamy je w `__slots__` podklasy.